In [1]:
import win32com.client as wc
import numpy as np

In [17]:
def static_mixer(radius, L_insert, h, file_name):
    '''This function creates just one helical insert of the static mixer. 
    radius - inner radius of the pipe
    L_insert - length of one insert
    h - thickness of the insert
    file_name - to save the file
    
    TO DO:
    function for smoothing the sweep guide rail
    mirror/pattern/loop for creation of the whole static mixer
    '''
    inv_app = wc.GetActiveObject("Inventor.Application")

    # Create a new part document
    part_doc = inv_app.Documents.Add(12290, inv_app.FileManager.GetTemplateFile(12290, 8962))
    #part_doc = inv_app.Documents.Add("Part", inv_app.FileManager.GetTemplateFile("Part"), True)
    comp_def = part_doc.ComponentDefinition
    
    # Create a 2D sketch for the tool body profile on the X-Y plane
    xy_plane = comp_def.WorkPlanes.Item(3) 
    profile_sketch = comp_def.Sketches.Add(xy_plane)
    radius = 1
    L_insert = 10
    # Define a rectangular profile for the tool body
    transient_geom = inv_app.TransientGeometry
    profile_sketch.SketchLines.AddAsTwoPointRectangle(
        transient_geom.CreatePoint2d(2*radius, h/2), 
        transient_geom.CreatePoint2d(-2*radius, -h/2) 
    )

    # Create a base feature from the profile to form the tool body
    profile = profile_sketch.Profiles.AddForSolid()
    
    cyl_sketch = comp_def.Sketches.Add(xy_plane)
    center = transient_geom.CreatePoint2d(0,0)
    cyl_sketch.SketchCircles.AddByCenterRadius(center, radius)

    cyl_profile = cyl_sketch.Profiles.AddforSolid()
    ext_solid_def = comp_def.Features.ExtrudeFeatures.CreateExtrudeDefinition(cyl_profile, 20481)
    ext_solid_def.SetDistanceExtent(L_insert, 20993)
    cyl_solid = comp_def.Features.ExtrudeFeatures.Add(ext_solid_def)
    
    plane = comp_def.Sketches.Add(comp_def.WorkPlanes.Item(3))

    comp_def.WorkPlanes.AddByPlaneAndOffset(plane, L_insert)
    
    sketch_mid = comp_def.Sketches.Add(comp_def.WorkPlanes.Item(4)) # new sketch
    
    sketch_mid.SketchLines.AddAsTwoPointRectangle(
    transient_geom.CreatePoint2d(2*radius, h/2),
    transient_geom.CreatePoint2d(-2*radius, -h/2))
    
    profile_mid = sketch_mid.Profiles.AddForSolid()
    
    rail_sketch = comp_def.Sketches3D.Add()
    oWPs1 = comp_def.WorkPoints.AddFixed(transient_geom.CreatePoint(2*radius, h/2, 0))
    oWPs2 = comp_def.WorkPoints.AddFixed(transient_geom.CreatePoint(-h/2, 2*radius, L_insert/2))
    oWPs3 = comp_def.WorkPoints.AddFixed(transient_geom.CreatePoint(-2*radius, -h/2, L_insert))
    oStartLine3d = rail_sketch.SketchLines3D.AddByTwoPoints(oWPs1, oWPs2)
    oLine3D = rail_sketch.SketchLines3D.AddByTwoPoints(oStartLine3d.EndSketchPoint, oWPs3)
    rail = comp_def.Features.CreatePath(oLine3D)
    
    path_sketch = comp_def.Sketches3D.Add()
    opath = path_sketch.SketchLines3D.AddByTwoPoints(
        transient_geom.CreatePoint(0, 0, 0),
        transient_geom.CreatePoint(0, 0, L_insert)
    )

    # Create the path object from the 3D sketch
    path = comp_def.Features.CreatePath(opath)
    
    sweep_def = comp_def.Features.SweepFeatures.CreateSweepDefinition(104450,profile, path, 20482)
    
    sweep_def.GuideRail = rail
    sweep_feature = comp_def.Features.SweepFeatures.Add(sweep_def)
    
    inv_app.ActiveDocument.SaveAs(f"D:\\Python Projects\\Inventor\\{file_name}.ipt", False)
    
    return None

In [19]:
geom = static_mixer(1, 1.5, 0.2, 'mixer1') 